<a href="https://colab.research.google.com/github/YoonHyeJu/project1/blob/master/Gradio_Sensitive_label.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive

In [3]:
drive.mount('/content/grive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/grive


In [5]:
import os
a=os.listdir('/content/grive/My Drive/Colab Notebooks/sensitive_label_detection/yolov3/data/data/images')
b=[]
for i in a:
    b.append(os.path.join('/content/grive/My Drive/Colab Notebooks/sensitive_label_detection/yolov3/data/data/images', i))
    
with open('/content/grive/My Drive/Colab Notebooks/sensitive_label_detection/yolov3/data/id.txt', 'w') as filehandle:
    
    for listitem in b:
        filehandle.write('%s\n' % listitem)

In [32]:
cd /content/grive/My Drive/Colab Notebooks/sensitive_label_detection/yolov3

/content/grive/My Drive/Colab Notebooks/sensitive_label_detection/yolov3


In [33]:
!pip install gradio 

In [34]:
!pip install doqu

In [35]:
!pip install utils

In [36]:
import argparse

In [37]:
from models import * # set ONNX_EXPORT in models.py
from utils.datasets import *
from utils.utils import *
import gradio as gr
import tensorflow as tf
import numpy as np
from PIL import Image
import requests
import cv2

In [42]:
#def sepia(radio, img):
def sepia(img):
  #checkbox_ = checkbox
  #connect model
  sepia_filter = np.array([[.393, .769, .189],
                           [.349, .686, .168],
                           [.272, .534, .131]])
  sepia_img = img.dot(sepia_filter.T)
  sepia_img /= sepia_img.max()                          
  return sepia_img

In [ ]:
def detect(source):
    inp = Image.fromarray(source.astype('uint8'), 'RGB')
    filename = "test.jpeg"
    inp.save(filename, "JPEG")

    cfg = '/content/grive/My Drive/Colab Notebooks/sensitive_label_detection/yolov3/cfg/yolov3-tiny.cfg'
    names = '/content/grive/My Drive/Colab Notebooks/sensitive_label_detection/yolov3/data/coco_id.names'
    weights = '/content/grive/My Drive/Colab Notebooks/sensitive_label_detection/yolov3/weights/last.pt'
    out = 'output'
    img_size=512
    conf_thres=0.3
    iou_thres=0.6
    fourcc='mp4v'
    half=False
    device=''
    view_img=False
    save_txt = False
    classes=None
    agnostic_nms=False
    augment=False
    cfg = check_file(cfg)
    names = check_file(names)
    save_img = True


    imgsz = (320, 192) if ONNX_EXPORT else img_size

    device = torch_utils.select_device(device='cpu' if ONNX_EXPORT else device)
    if os.path.exists(out):
        shutil.rmtree(out)
    os.makedirs(out)

    model = Darknet(cfg, imgsz)

    attempt_download(weights)
    if weights.endswith('.pt'):
        model.load_state_dict(torch.load(weights, map_location=device)['model'])
    else:
        load_darknet_weights(model, weights)

    classify = False
    if classify:
        modelc = torch_utils.load_classifier(name='resnet101', n=2)  # initialize
        modelc.load_state_dict(torch.load('weights/resnet101.pt', map_location=device)['model'])  # load weights
        modelc.to(device).eval()

    # Eval mode
    model.to(device).eval()
    
    # Fuse Conv2d + BatchNorm2d layers
    # model.fuse()

    # Export mode
    if ONNX_EXPORT:
        model.fuse()
        img = torch.zeros((1, 3) + imgsz)  # (1, 3, 320, 192)
        f = weights.replace(weights.split('.')[-1], 'onnx')  # *.onnx filename
        torch.onnx.export(model, img, f, verbose=False, opset_version=11,
                          input_names=['images'], output_names=['classes', 'boxes'])

        # Validate exported model
        import onnx
        model = onnx.load(f)  # Load the ONNX model
        onnx.checker.check_model(model)  # Check that the IR is well formed
        print(onnx.helper.printable_graph(model.graph))  # Print a human readable representation of the graph
        return

    # Half precision
    half = half and device.type != 'cpu'  # half precision only supported on CUDA
    if half:
        model.half()

    # Set Dataloader
    vid_path, vid_writer = None, None
    #############
    dataset = LoadImages(filename, img_size=imgsz)
    names=load_classes(names)
    colors = [[random.randint(0, 255) for _ in range(3)] for _ in range(len(names))]

    # Run inference
    t0 = time.time()
    img = torch.zeros((1, 3, imgsz, imgsz), device=device)  # init img
    _ = model(img.half() if half else img.float()) if device.type != 'cpu' else None  # run once
    for path, img, im0s, vid_cap in dataset:
        img = torch.from_numpy(img).to(device)
        img = img.half() if half else img.float()  # uint8 to fp16/32
        img /= 255.0  # 0 - 255 to 0.0 - 1.0
        if img.ndimension() == 3:
            img = img.unsqueeze(0)

        # Inference
        t1 = torch_utils.time_synchronized()
        pred = model(img, augment=augment)[0]
        t2 = torch_utils.time_synchronized()

        # to float
        if half:
            pred = pred.float()

        # Apply NMS
        pred = non_max_suppression(pred, conf_thres, iou_thres,
                                   multi_label=False, classes=classes, agnostic=agnostic_nms)
        
        # Apply Classifier
        if classify:
            pred = apply_classifier(pred, modelc, img, im0s)

        # Process detections
        for i, det in enumerate(pred):  # detections for image i
            p, s, im0 = path, '', im0s

            save_path = str(Path(out) / Path(p).name)
            s += '%gx%g ' % img.shape[2:]  # print string
            gn = torch.tensor(im0.shape)[[1, 0, 1, 0]]  #  normalization gain whwh
            if det is not None and len(det):
                # Rescale boxes from imgsz to im0 size
                det[:, :4] = scale_coords(img.shape[2:], det[:, :4], im0.shape).round()

                # Print results
                for c in det[:, -1].detach().unique():
                    n = (det[:, -1] == c).sum()  # detections per class
                    s += '%g %ss, ' % (n, names[int(c)])  # add to string

                # Write results
                for *xyxy, conf, cls in reversed(det):
                    if save_txt:  # Write to file
                        xywh = (xyxy2xywh(torch.tensor(xyxy).view(1, 4)) / gn).view(-1).tolist()  # normalized xywh
                        with open(save_path[:save_path.rfind('.')] + '.txt', 'a') as file:
                            file.write(('%g ' * 5 + '\n') % (cls, *xywh))  # label format

                    if save_img or view_img:  # Add bbox to image
                        label = '%s %.2f' % (names[int(cls)], conf)
                        plot_one_box(xyxy, im0, label=label, color=colors[int(cls)])

            # Print time (inference + NMS)
            print('%sDone. (%.3fs)' % (s, t2 - t1))

            # Stream results
            if view_img:
                cv2.imshow(p, im0)
                if cv2.waitKey(1) == ord('q'):  # q to quit
                    raise StopIteration

            # Save results (image with detections)
            if save_img:
                if dataset.mode == 'images':
                    cv2.imwrite(save_path, im0)
                else:
                    if vid_path != save_path:  # new video
                        vid_path = save_path
                        if isinstance(vid_writer, cv2.VideoWriter):
                            vid_writer.release()  # release previous video writer

                        fps = vid_cap.get(cv2.CAP_PROP_FPS)
                        w = int(vid_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
                        h = int(vid_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
                        vid_writer = cv2.VideoWriter(save_path, cv2.VideoWriter_fourcc(*fourcc), fps, (w, h))
                    vid_writer.write(im0)

    if save_txt or save_img:
        print('Results saved to %s' % os.getcwd() + os.sep + out)
        if platform == 'darwin':  # MacOS
            os.system('open ' + save_path)

    print('Done. (%.3fs)' % (time.time() - t0))
    print('save path: ')
    out_path=os.getcwd() + os.sep + save_path
    return out_path

image = gr.inputs.Image()
label = gr.outputs.Image(label = 'Sensitive Label Detection', type='file')
gr.Interface(fn=detect, inputs=image, outputs=label, capture_session=True).launch(debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on External URL: https://16001.gradio.app
Interface loading below...


Using CPU

Model Summary: 37 layers, 8.07261e+06 parameters, 8.07261e+06 gradients
image 1/1 test.jpeg: 512x384 3 sensitive_labels, Done. (0.177s)
Results saved to /content/grive/My Drive/Colab Notebooks/sensitive_label_detection/yolov3/output
Done. (0.263s)
save path: 
Using CPU

Model Summary: 37 layers, 8.07261e+06 parameters, 8.07261e+06 gradients
image 1/1 test.jpeg: 512x384 3 sensitive_labels, Done. (0.179s)
Results saved to /content/grive/My Drive/Colab Notebooks/sensitive_label_detection/yolov3/output
Done. (0.277s)
save path: 
